https://www.electionstudies.org/wp-content/uploads/2017/02/anes_timeseries_cdf_codebook_var.txt

https://electionstudies.org/anes_timeseries_cdf/

In [1]:
import modpipe
import re
from collections import OrderedDict
import os

In [2]:
VERSION_RE = re.compile("^RELEASE VERSION:\s+(\d+)")


LINE_SEP = "=" * 77 + "\n"

def defn_iterator(file_path):
    version = None
    lines = []
    
    with open(file_path, errors='ignore') as fp:
        
        for line in fp:
            if version is None:
                version = VERSION_RE.search(line).group(1)
                yield {'version': version}
            elif line == LINE_SEP:
                break
        
        for line in fp:
            if line == LINE_SEP:
                yield lines
                lines = []
            else:
                lines.append(line.rstrip())
                
    if lines[-1] == "" and lines[-2] == "" and lines[-3] == "1":
        yield lines[:-3]

In [10]:
DATA_PATH = os.path.join("data", "raw", "anes_timeseries_cdf_codebook_var.txt")

general_notes, var_defs = [], OrderedDict()

with modpipe.ModPipe("codebook_pipeline") as pipe:
    for lines in defn_iterator(DATA_PATH):
        if 'version' not in lines:
            res = pipe(lines.copy())
            if '_general_note_lines' in res:
                general_notes.append(res)
            else:
                var_defs[res['name']] = res

In [11]:
var_defs['VCF9074']

OrderedDict([('name', 'VCF9074'),
             ('module', 'ISSUES'),
             ('desc', 'President on Cooperation with USSR Scale'),
             ('var_type', 'Numeric  Dec 0-1'),
             ('prompt',
              ['1980:',
               'Some people feel it is important for us to try very hard to get along',
               'with Russia.  Others feel it is a big mistake to try too hard to get',
               'along with Russia.',
               '1984,1988:',
               'Some people feel it is important for us to try to cooperate more with',
               'Russia, while others believe we should be much tougher in our dealings',
               'with Russia.',
               'ALL YEARS:',
               'Where would you place [president] on this scale?  (7-POINT SCALE SHOWN',
               'TO R)']),
             ('source_vars',
              OrderedDict([('1980', ['V80107']),
                           ('1984', ['V84040']),
                           ('1988', ['V88037'])])